Importing Libraries 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error

Loading all the datasets 

In [ ]:
train=pd.read_csv('D:/Kaggle/Predict Future Sales/sales_train.csv')
test=pd.read_csv('D:/Kaggle/Predict Future Sales/test.csv')
itm_cat=pd.read_csv('D:/Kaggle/Predict Future Sales/item_categories.csv')
itms=pd.read_csv('D:/Kaggle/Predict Future Sales/items.csv')
shops=pd.read_csv('D:/Kaggle/Predict Future Sales/shops.csv')
train.head()

In [ ]:
test.head()

Checking for NaNs

In [ ]:
train.isnull().sum()

Modification of the date feature as Prediction is expected by months

In [ ]:
train['date']=pd.to_datetime(train['date'])
train['date']=train['date'].apply(lambda strp: strp.strftime('%Y-%m'))

Dropping these columns as it won't be necessary since these are not independent feature to predict the test data.

In [ ]:
train.drop(columns=['date_block_num','item_price'],inplace=True)

In [ ]:
train.head()

In [ ]:
train.tail()

Transforming the table

In [ ]:
new_train=train.groupby(['date','shop_id','item_id']).sum()
new_train=new_train.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
new_train.reset_index(inplace=True)
pd.set_option('display.max_columns',40)
new_train.head()

Now we perform left join of the test data refering to the shop and item id feature keeping the unpredicted test data as 0

In [ ]:
test=pd.merge(test,new_train,on=['shop_id','item_id'],how='left')
test.drop(['ID','2013-01'],axis=1,inplace=True)
test=test.fillna(0)
test.head()

Perdorming Train test split 

In [ ]:
ytrain=new_train['2015-10'].values
xtrain=new_train.drop(['2015-10'],axis = 1)
xtest=test

In [ ]:
xtrain,xtest,ytrain,ytest=tts(xtrain,ytrain,test_size=0.2,random_state=101)

Linear Models Linear,Lasso and Ridge Regression

In [ ]:
lr_model=LinearRegression(normalize=True).fit(xtrain,ytrain)
lr_y_pred=lr_model.predict(xtest)
print('Linear Training  mse:',mean_squared_error(ytrain,lr_model.predict(xtrain)))
print('Linear Test mse:',mean_squared_error(ytest,lr_model.predict(xtest)))
print('Linear Test score:',lr_model.score(xtrain,ytrain))
lasso_model=Lasso(alpha=0.1).fit(xtrain,ytrain)
ls_y_pred=lasso_model.predict(xtest)
print('Lasso Training  mse:',mean_squared_error(ytrain,lasso_model.predict(xtrain)))
print('Lasso Test mse:',mean_squared_error(ytest,lasso_model.predict(xtest)))
print('Lasso Test score:',lasso_model.score(xtrain,ytrain))
ridge_model=Ridge(alpha=0.5).fit(xtrain,ytrain)
rd_y_pred=ridge_model.predict(xtest)
print('Ridge Training  mse:',mean_squared_error(ytrain,ridge_model.predict(xtrain)))
print('RidgeTest mse:',mean_squared_error(ytest,ridge_model.predict(xtest)))
print('Ridge Test score:',ridge_model.score(xtrain,ytrain))

Ensembles Random Forest and Gradient Boosting

In [ ]:
rfr_model=RandomForestRegressor(n_estimators=100,criterion='mse').fit(xtrain,ytrain)
rfr_y_pred=rfr_model.predict(xtest)
print('Random Forest Training  mse:',mean_squared_error(ytrain,rfr_model.predict(xtrain)))
print('Random Forest Test mse:',mean_squared_error(ytest,rfr_model.predict(xtest)))
print('Random Forest Test score:',rfr_model.score(xtrain,ytrain))
gbr_model=GradientBoostingRegressor(n_estimators=100,criterion='mse').fit(xtrain,ytrain)
gbr_y_pred=gbr_model.predict(xtest)
print('GBR Training  mse:',mean_squared_error(ytrain,gbr_model.predict(xtrain)))
print('GBR Test mse:',mean_squared_error(ytest,gbr_model.predict(xtest)))
print('GBR Test score:',gbr_model.score(xtrain,ytrain))

In [ ]:
prediction=list(map(round,gbr_y_pred))
final_test=pd.read_csv('D:/Kaggle/Predict Future Sales/test.csv')
solution=pd.DataFrame({'item_cnt_day':prediction})
solution_final=pd.concat([final_test,solution],axis=1)
solution_final.head()

In [ ]:
solution_file=solution_final.to_csv('D:/Kaggle/Predict Future Sales/submission.csv')